In [1]:
import numpy as np
import pandas as pd
import cv2
import random
from sklearn.metrics import confusion_matrix
from sklearn.pipeline import make_pipeline
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn import metrics
from mostCommon import most_common
from sklearn.metrics import f1_score
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import math

In [2]:
my_data = pd.read_csv('../data/Five_Frames_Per_patient.csv')
patients_data=pd.read_csv('../data/videos_data.csv')
data_array = my_data.to_numpy()
patients_array=patients_data["FileName"].to_numpy()
images_path = '../data/Five_Frames_Per_patient/'
no_of_test_patients=math.floor(patients_array.size*0.2)
images_data = []
models = []

In [3]:
rand_indexes=[]
while len(rand_indexes)<no_of_test_patients:
    h=random.randint(0, 121)
    if h not in rand_indexes:
        rand_indexes.append(h)
test_patients=np.take(patients_array,rand_indexes)
print(test_patients)

['Reg_Avi_Video_004' 'Cov_denault_proposedUS_vid1'
 'Reg_northumbria_0409_set5_vid1' 'Cov_denault_proposedUS_vid2'
 'Cov_convex_volpecelli_sonographic_v5' 'Reg_pat3Image_135128'
 'Cov_recommendations_lightbeam_mov6' 'Pneu_Basal-pneumonia-RUQ_crop'
 'Pneu_northumbria_0409_set1_vid1' 'Cov-Atlas-+(43)'
 'Pneu_northumbria_0409_set5_vid1' 'Cov-Atlas+(44)'
 'Reg_Image_182340_crop' 'Pneu_northumbria_0409_set6_vid7'
 'Pneu_northumbria_0409_set1_vid2' 'Reg_northumbria_1009_vid2'
 'Pneu_Avi_Video_004' 'Pneu-Atlas-pneumonia2' 'Pneu_from_article_rippey'
 'Reg_pat3Image_134904' 'Reg_northumbria_1009_vid3' 'Reg_pat1Image_132943'
 'Reg_Avi_Video_003' 'Pneu_Avi_Video_002']


In [4]:
#I got my patients names now i need to get their frames from my_data

df=my_data
test_data=[]
for i in range(len(test_patients)):
    contain_values = df[df['FileName'].str.contains(test_patients[i])].to_numpy()
    test_data.append(contain_values)
    print(contain_values)
    indexes = df[df['FileName'].str.contains(test_patients[i])].index
    df.drop(indexes, inplace = True )

[['Reg_Avi_Video_004_p81_f1' 'regular' 'jpg']
 ['Reg_Avi_Video_004_p81_f2' 'regular' 'jpg']
 ['Reg_Avi_Video_004_p81_f3' 'regular' 'jpg']
 ['Reg_Avi_Video_004_p81_f4' 'regular' 'jpg']
 ['Reg_Avi_Video_004_p81_f5' 'regular' 'jpg']]
[['Cov_denault_proposedUS_vid10_p20_f1' 'covid' 'jpg']
 ['Cov_denault_proposedUS_vid10_p20_f2' 'covid' 'jpg']
 ['Cov_denault_proposedUS_vid10_p20_f3' 'covid' 'jpg']
 ['Cov_denault_proposedUS_vid10_p20_f4' 'covid' 'jpg']
 ['Cov_denault_proposedUS_vid10_p20_f5' 'covid' 'jpg']
 ['Cov_denault_proposedUS_vid11_p21_f1' 'covid' 'jpg']
 ['Cov_denault_proposedUS_vid11_p21_f2' 'covid' 'jpg']
 ['Cov_denault_proposedUS_vid11_p21_f3' 'covid' 'jpg']
 ['Cov_denault_proposedUS_vid11_p21_f4' 'covid' 'jpg']
 ['Cov_denault_proposedUS_vid11_p21_f5' 'covid' 'jpg']
 ['Cov_denault_proposedUS_vid12_p22_f1' 'covid' 'jpg']
 ['Cov_denault_proposedUS_vid12_p22_f2' 'covid' 'jpg']
 ['Cov_denault_proposedUS_vid12_p22_f3' 'covid' 'jpg']
 ['Cov_denault_proposedUS_vid12_p22_f4' 'covid' 'jpg']

C:\Users\RS3\AppData\Roaming\Python\Python39\site-packages\pandas\core\strings\accessor.py:101: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)


In [6]:
X_train=[]
X_test=[]
y_train=[]
y_test=[]

In [7]:
for i in data_array:
    img = cv2.imread(images_path + i[0] + '.' + i[2], cv2.IMREAD_GRAYSCALE)
    img = cv2.equalizeHist(img)
    img = cv2.resize(img, (224, 224), interpolation=cv2.INTER_AREA)
    img_1d = img.reshape(224 * 224)
    images_data.append(img_1d)
X_train=np.asarray(images_data)
y = np.asarray(my_data['Label'])

In [8]:
patients_images=[]
for patient in test_data:
    for i in patient:
        img = cv2.imread(images_path + i[0] + '.' + i[2], cv2.IMREAD_GRAYSCALE)
        img = cv2.equalizeHist(img)
        img = cv2.resize(img, (224, 224), interpolation=cv2.INTER_AREA)
        img_1d = img.reshape(224 * 224)
        X_test.append(img_1d)
        y_test.append(i[1])

In [45]:
# Now I have xtrain xtest ytrain ytest ready I should increment them in X
X_test
y_test

['covid',
 'covid',
 'covid',
 'covid',
 'covid',
 'covid',
 'covid',
 'covid',
 'covid',
 'covid',
 'regular',
 'regular',
 'regular',
 'regular',
 'regular',
 'pneumonia',
 'pneumonia',
 'pneumonia',
 'pneumonia',
 'pneumonia',
 'regular',
 'regular',
 'regular',
 'regular',
 'regular',
 'regular',
 'regular',
 'regular',
 'regular',
 'regular',
 'regular',
 'regular',
 'regular',
 'regular',
 'regular',
 'covid',
 'covid',
 'covid',
 'covid',
 'covid',
 'regular',
 'regular',
 'regular',
 'regular',
 'regular',
 'regular',
 'regular',
 'regular',
 'regular',
 'regular',
 'covid',
 'covid',
 'covid',
 'covid',
 'covid',
 'pneumonia',
 'pneumonia',
 'pneumonia',
 'pneumonia',
 'pneumonia',
 'regular',
 'regular',
 'regular',
 'regular',
 'regular',
 'regular',
 'regular',
 'regular',
 'regular',
 'regular',
 'covid',
 'covid',
 'covid',
 'covid',
 'covid',
 'covid',
 'covid',
 'covid',
 'covid',
 'covid',
 'pneumonia',
 'pneumonia',
 'pneumonia',
 'pneumonia',
 'pneumonia',
 'pneumoni

In [46]:
r = []
conf_matricies = []
f1_reports = []
f1_micro = []
for i in range(3):
    r.append(random.randint(0, 1000))
print('Random seeds are: '+str(r))

number_of_models = 11
X = np.asarray(images_data)
y = np.asarray(my_data['Label'])

for k in range(len(r)):
    print('--------------------------------- Iteration ' + str(k) + '--------------------------------- \n')
    Xandy = []
    models = []
    # creating models
    for i in range(number_of_models):
        models.append(make_pipeline(StandardScaler(), SVC(gamma='auto', random_state=r[k])))

    # Creating array test splits for models
    # X_train, X_test, y_train, y_test
    for i in range(number_of_models):
        x = train_test_split(X, y, test_size=0.2, random_state=r[k])
        Xandy.append(x)

    print('Equalizing Data...')
    for i in range(number_of_models):
        x = Xandy[i][0]
        y1 = Xandy[i][2]
        cov = np.count_nonzero(y1 == 'covid')
        pneu = np.count_nonzero(y1 == 'pneumonia')
        reg = np.count_nonzero(y1 == 'regular')
        # deleting random pneumonia elements from each model set
        while cov != pneu:
            index = random.randint(0, len(x) - 1)
            if y1[index] == 'pneumonia':
                x = np.delete(x, index, 0)
                y1 = np.delete(y1, index, 0)
                pneu -= 1
        # deleting random regular elements from each model set
        while cov != reg:
            index = random.randint(0, len(x) - 1)
            if y1[index] == 'regular':
                x = np.delete(x, index, 0)
                y1 = np.delete(y1, index, 0)
                reg -= 1
        Xandy[i][0] = x
        Xandy[i][2] = y1

    print('Normalizing and applying PCA...')
    for i in range(number_of_models):
        scaler = StandardScaler()
        # Fit on training set only
        scaler.fit(Xandy[i][0])

        # Apply transform on both training and test set
        Xandy[i][0] = scaler.transform(Xandy[i][0])
        Xandy[i][1] = scaler.transform(Xandy[i][1])
    for i in range(number_of_models):
        pca = PCA(.95)
        pca.fit(Xandy[i][0])
        Xandy[i][0] = pca.transform(Xandy[i][0])
        Xandy[i][1] = pca.transform(Xandy[i][1])
    print(len(Xandy[0][0][0]))
    print('Training Models...')
    for i in range(number_of_models):
        models[i].fit(Xandy[i][0], Xandy[i][2])

    print('getting predictions of each model...')
    predicted = []
    for i in range(number_of_models):
        predicted.append(models[i].predict(Xandy[i][1]))

    voted_prediction = []
    for i in range(len(predicted[0])):
        voting = []
        for j in range(len(predicted)):
            voting.append(predicted[j][i])
        voted_prediction.append(most_common(voting))
    conf_matrix = confusion_matrix(Xandy[0][3], voted_prediction)
    print(conf_matrix)
    conf_matricies.append(conf_matrix)

    classification_report = metrics.classification_report(Xandy[0][3], voted_prediction, digits=3)
    print(classification_report)
    f1_reports.append(classification_report)
    f1_micro.append(f1_score(Xandy[0][3], voted_prediction, average='micro'))

print('Average f1 micro of all iterations is: ' + str(sum(f1_micro) / len(f1_micro)))

print('Saving models....')
np.save('models', models)
print('All Done :)')

Random seeds are: [242, 35, 203]
--------------------------------- Iteration 0--------------------------------- 



ValueError: Found input variables with inconsistent numbers of samples: [1220, 480]